<a href="https://colab.research.google.com/github/chanlenium/Python/blob/main/DataAnalytics/RandomForest_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

< 랜덤포레스트를 이용한 예측 문제 해결 >
- 다수의 의사결정 트리들을 Bagging(같은 종류의 알고리즘 모델을 여러 개 결합하여 예측하는 방법)하여 회귀를 수행
- 각 트리는 전체 학습 데이터 중 서로 다른 데이터를 샘플링하여 일부 데이터를 제외한 후 최적의 특징을 찾아 트리를 분기함
- 트리 모델의 개수(n_estimators)와 개별 트리의 깊이(max_depth) 매개변수 값을 잘 조절하여 예측의 정확도를 높임

(분석 목표 예) 캘리포니아 인구가구 통계 데이터셋에서 주택중위가치(median_house_value)에 영향을 주는 변수를 찾아보고, 다중선형회구모델을 생성&평가함
- 종속변수 : median_house_value

In [2]:
# 분석에 필요한 패키지 임포트
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt

# 랜덤포레스트 모델을 위한 패키지
from sklearn.ensemble import RandomForestRegressor

# 학습/테스트 데이터셋 분리를 위한 패키지
from sklearn.model_selection import train_test_split

In [3]:
# 데이터 불러오기
df = pd.read_csv("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv")
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [4]:
# 데이터프레임의 info()함수를 사용하여 데이터셋에 결측값 존재 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [5]:
# 데이터프레임의 기술통계를 보여줌
df.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [6]:
# 데이터 전처리
# total_bedrooms 컬럼은 결측데이터가 있기 때문에 결측데이터가 있는 행 전체를 제거함
# "ocean_proximity"는 범주형 값이므로, 분석에서 제외함

# 결측값이 있는 행 전체(axis = 0)를 제거
df = df.dropna(axis = 0)

# "ocean_prozimity" 컬럼 제거
df = df.drop("ocean_proximity", axis = 1)
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


In [8]:
# 상관관계 분석 
corr = df.corr(method = "pearson")
corr

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
longitude,1.000000,-0.924616,-0.109357,0.045480,0.069608,0.100270,0.056513,-0.015550,-0.045398
latitude,-0.924616,1.000000,0.011899,-0.036667,-0.066983,-0.108997,-0.071774,-0.079626,-0.144638
housing_median_age,-0.109357,0.011899,1.000000,-0.360628,-0.320451,-0.295787,-0.302768,-0.118278,0.106432
total_rooms,0.045480,-0.036667,-0.360628,1.000000,0.930380,0.857281,0.918992,0.197882,0.133294
total_bedrooms,0.069608,-0.066983,-0.320451,0.930380,1.000000,0.877747,0.979728,-0.007723,0.049686
population,0.100270,-0.108997,-0.295787,0.857281,0.877747,1.000000,0.907186,0.005087,-0.025300
households,0.056513,-0.071774,-0.302768,0.918992,0.979728,0.907186,1.000000,0.013434,0.064894
median_income,-0.015550,-0.079626,-0.118278,0.197882,-0.007723,0.005087,0.013434,1.000000,0.688355
median_house_value,-0.045398,-0.144638,0.106432,0.133294,0.049686,-0.025300,0.064894,0.688355,1.000000


In [9]:
# 분석 데이터셋 준비
# median_house_value를 제외한 나머지를 독립변수로 함
X = df.drop("median_house_value", axis = 1)
y = df["median_house_value"]

In [10]:
# 분석데이터셋 분할(7:3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 11)

In [11]:
print(X_train.shape)  # 학습 독립변수
print(X_test.shape) # 테스트 독립변수
print(y_train.shape)  # 학습 종속변수
print(y_test.shape) # 테스트 종속변수

(14303, 8)
(6130, 8)
(14303,)
(6130,)


In [12]:
# 데이터 분석 수행
# RandomForestRegressor 객체 생성
rfr = RandomForestRegressor(max_depth = 3, random_state = 42)
# 학습 수행
rfr.fit(X_train, y_train)

RandomForestRegressor(max_depth=3, random_state=42)

In [13]:
# 학습이 완료된 dtr 객체에서 테스트 데이터셋으로 예측 수행
pred = rfr.predict(X_test)
pred

array([153506.2352407 , 389701.21674272, 153506.2352407 , ...,
       116154.73094311, 292480.25277903, 155162.84054196])

In [14]:
# 모델 성능 평가 - 테스트 데이터셋
# MSE 값ㅇ르 구해서 모델의 성능을 평가
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, pred)
mse

6477478403.694343

In [16]:
# 학습이 완료된 dtr 객체에서 학습(트레이닝) 데이터셋으로 예측 수행
pred = rfr.predict(X_train)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_train, pred)
mse

6351748616.32075